In [1]:
import torch
from torch_harmonics import DiscreteContinuousConvS2

In [2]:
layer = DiscreteContinuousConvS2(
    in_channels=3,
    out_channels=5,
    in_shape=(180,360),
    out_shape=(180,360),
    kernel_shape=(3,3),
)

In [4]:
x = torch.rand(2,3,180,360)
x = layer(x)
x.shape

torch.Size([2, 5, 180, 360])